In [1]:
import pandas as pd
import tesis_orm
from utilidades_tesis import normalizar_titulo, comparar_difuso, buscar_parecidos

C:\Users\Enrique\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
limpios = pd.read_excel("Data/tesis_limpias2.xlsx")
limpios["C.I"] = limpios["C.I"].fillna(value=0).astype(int).astype(str).replace("0", "")
limpios.to_excel("Data/tesis_limpias2.xlsx")
limpios.head()

,TÍTULO,LINEA_INVESTIGACION,NOMBRE_ALUMNO,APELLLIDO_ALUMNO,C.I,NOMBRE_TUTOR,APELLIDO_TUTOR,TUTOR_CEDULA
65,DESARROLLO DE UN SISTEMA QUE GENERE REPORTE 14...,Ingeniería de Software,ELMER,MORALES,1,CARINA,GARCIA,3
77,DESARROLLO DE UN SISTEMA DE INFORMACION PARA E...,Ingeniería de Software,ALEJANDRO,VALERO,2,CARINA,GARCIA,3
82,DESARROLLO DE UN SISTEMA DE INFORMACIÓN QUE GE...,Ingeniería de Software,JUAN,VIEIRO,3,CARINA,GARCIA,3
7,DESARROLLAR UN SISTEMA DE HEPL DESK QUE PERMIT...,Ingeniería de Software,LUIS,DÍAZ,4,MARILENA,TOZZI,21
35,DESARROLLO DE UN SISTEMA DE INFORMACIÓN PARA E...,Ingeniería de Software,LERVIS,NIÑO,5,DAVID,RAÍCES,5


In [3]:
bd = tesis_orm.conectar_a_bd()
bd

In [4]:
def guardar_tesis_en_bd(tesis_data):
    # Alumno
    # Revisa si existe el autor de esta tesis
    alumnos = bd.query(tesis_orm.Alumno).filter_by(nombre=tesis_data[2]).filter_by(apellido=tesis_data[3]).first()
    if alumnos:
        alumno = alumnos
    else:
        # Si no existe, guardalo en BD
        alumno = tesis_orm.Alumno(
            nombre=tesis_data[2],
            apellido=tesis_data[3],
            cedula=tesis_data[4]
        )
        bd.add(alumno)
    
    # Tutor
    # Revisa si existes el tutor de esta tesis
    tutor = bd.query(tesis_orm.Tutor).filter_by(cedula=tesis_data[7]).first()
    if not tutor:
        # Si no existe, guardalo en BD
        tutor = tesis_orm.Tutor(
            nombre=tesis_data[5],
            apellido=tesis_data[6],
            cedula=tesis_data[7]
        )
        bd.add(tutor)
    
    # Linea de investigacion
    # Revisa si existe la linea de investigacion
    l_inv = bd.query(tesis_orm.LineaDeInvestigacion).filter_by(nombre=tesis_data[1]).first()
    if not l_inv:
        # Si no existe, guardala en BD
        l_inv = tesis_orm.LineaDeInvestigacion(nombre=tesis_data[1])
        
    # Tesis   
    # Asigna tutor, autor y linea de investigacion a la tesis
    t = tesis_orm.Tesis()
    t.titulo = tesis_data[0]
    t.tutor = tutor
    t.titulo_normalizado = tesis_data[8]
    t.autor = alumno
    t.linea_investigacion = l_inv
    
    bd.add(t)

In [5]:
limpios["titulo_normalizado"] = limpios["TÍTULO"].apply(normalizar_titulo)
limpios.head()

,TÍTULO,LINEA_INVESTIGACION,NOMBRE_ALUMNO,APELLLIDO_ALUMNO,C.I,NOMBRE_TUTOR,APELLIDO_TUTOR,TUTOR_CEDULA,titulo_normalizado
65,DESARROLLO DE UN SISTEMA QUE GENERE REPORTE 14...,Ingeniería de Software,ELMER,MORALES,1,CARINA,GARCIA,3,desarroll sistem gener report 00 - 000 bas sis...
77,DESARROLLO DE UN SISTEMA DE INFORMACION PARA E...,Ingeniería de Software,ALEJANDRO,VALERO,2,CARINA,GARCIA,3,desarroll sistem informacion control gestion t...
82,DESARROLLO DE UN SISTEMA DE INFORMACIÓN QUE GE...,Ingeniería de Software,JUAN,VIEIRO,3,CARINA,GARCIA,3,desarroll sistem informacion gestion report in...
7,DESARROLLAR UN SISTEMA DE HEPL DESK QUE PERMIT...,Ingeniería de Software,LUIS,DÍAZ,4,MARILENA,TOZZI,21,desarroll sistem hepl desk permit optimiz rend...
35,DESARROLLO DE UN SISTEMA DE INFORMACIÓN PARA E...,Ingeniería de Software,LERVIS,NIÑO,5,DAVID,RAÍCES,5,desarroll sistem informacion registr control h...


In [6]:
for fila in limpios.iterrows():
    guardar_tesis_en_bd(fila[1])
    
bd.commit()